In [1]:
notebookName = 'audio-deepfake-detection-training'
runJobId = 'ASVspoof-2019_training'
# runJobId = 'ASVspoof-2019_small-eval-1'     # This should fail

import configuration.configuration as configuration
import model_definitions.model_cnn_definition as model_cnn_definition
from mel_spectrogram.mel_spectrogram import MelSpectrogramGenerator
from notebook_utils import notebookToPython
from processors.basic_model_training_processor import BasicModelTrainingProcessor
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor


In [2]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

if (job.newModelGenerated == False):
    raise ValueError("This notebook is meant for training. Select a job without a value for 'persisted-model' set.")

Write python file
Generating new model name: output/ASVspoof-2019_training_2025-03-22T12-19-50.794437.libjob
Assigned model name: output/ASVspoof-2019_training_2025-03-22T12-19-50.794437.libjob


In [3]:
generator = MelSpectrogramGenerator()
X, y_encoded = generator.generateMelSpectrograms(job, job.dataPathSuffix)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt...
fullDataPath: C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac
Loading audio files: 1269
Loading audio files: 2538
Loading audio files: 3807
Loading audio files: 5076
Loading audio files: 6345
Loading audio files: 7614
Loading audio files: 8883
Loading audio files: 10152
Loading audio files: 11421
Loading audio files: 12690
Loading audio files: 13959
Loading audio files: 15228
Loading audio files: 16497
Loading audio files: 17766
Loading audio files: 19035
Loading audio files: 20304
Loading audio files: 21573
Loading audio files: 22842
Loading audio files: 24111
Loading audio files: 25380
Number of audio files load: 25380


In [4]:
trainingProc = BasicModelTrainingProcessor(job, model_cnn_definition.ModelCnnDefinition)
model, X_train, X_test, y_train, y_test = trainingProc.process(X, y_encoded, 1)

Selecting training and test data
Training using 20304 files.
    def buildModel(self) -> Model:
        print(f"__job__: {self.__job__}")
        input_shape = (self.__job__.numMels, self.width, self.channels)
        model_input = Input(shape = input_shape)

        # TODO - why were these parameters selected? What purpose do they serve? Should they be configurable?
        x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(model_input)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Flatten()(x)
        x = Dense(units=128, activation='relu')(x)
        x = Dropout(0.5)(x)

        model_output = Dense(self.__job__.numClasses, activation='softmax')(x)

        return Model(inputs=model_input, outputs=model_output)

Model definition:
None
__job__: <configuration.configuration.Job object at 0x00000192D694D610>
Training the Model...
Epoch 1/10
635/6

### Test Model

In [5]:
evaluationProc = BasicModelEvaluationProcessor(job, model)
evaluationProc.process(X_test, y_test)

print("\n")
evaluationProc.reportSnapshot()

159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step
  Batches: 1 - Files: 5076 - Score: 0.9753743104806935 - Elements: 5076


start time: 2025-03-22T17:31:18.745079+00:00
end time: 2025-03-22T17:31:21.602095+00:00
elapsed: 0:00:02.857016

model file: output/ASVspoof-2019_training_2025-03-22T12-19-50.794437.libjob
batch count: 1
file count: 5076
accuracy_score: 0.9753743104806935

job: {
    "jobId": "ASVspoof-2019_training",
    "inputFileBatchSize": 1000,
    "outputFolder": "output",
    "dataPathRootRaw": "$HOMEDRIVE$HOMEPATH/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathRoot": "C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019",
    "dataPathSuffix": "LA/ASVspoof2019_LA_train/flac",
    "dataExtension": ".flac",
    "labelFilename": "LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt",
    "executeToCategoricalForLabels": true,
    "numClasses": 2,
    "sampleRate": 16000,
    "duration": 5,
    "numMels": 128,
    "maxTimeSteps": 109,
    "optimizer": "adam",
    "lo